In [ ]:
import cdsapi

dataset = "reanalysis-era5-land"
request = {
    "variable": ["lake_ice_temperature"],
    "year": "2024",
    "month": "01",
    "day":   "01",
    "time":  ["00:00"],
    "format": "netcdf",
    "area": [39.5, 124, 33, 131
    ],
}
client = cdsapi.Client()
client.retrieve(dataset, request).download()

2025-05-12 18:03:56,503 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-12 18:03:56,504 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-12 18:03:57,456 INFO Request ID is 29d12471-44a6-435e-86c9-953b52a33203
2025-05-12 18:03:57,809 INFO status has been updated to accepted
2025-05-12 18:04:15,981 INFO status has been updated to running
2025-05-12 18:04:23,914 INFO status has been updated to successful


50362cb74fcf296ea5cb5ccfccba6d1f.zip:   0%|          | 0.00/28.3k [00:00<?, ?B/s]

'50362cb74fcf296ea5cb5ccfccba6d1f.zip'

In [14]:
import xarray as xr
import pandas as pd

# 1. NetCDF 파일 열기
ds = xr.open_dataset(r"C:\Users\USER\Desktop\my_git\fire\50362cb74fcf296ea5cb5ccfccba6d1f\data_0.nc")

# 2. Dataset 전체를 DataFrame으로 변환
df = ds.to_dataframe().reset_index()

# 3. CSV로 저장
df.to_csv("output.csv", index=False)


In [15]:
import cdsapi
c = cdsapi.Client()
c.retrieve(
    'reanalysis-era5-land',
    {
        'variable': [
            '2m_dewpoint_temperature',  # d2m
            'total_precipitation',      # tp
            '2m_temperature',           # t2m
            '10m_u_component_of_wind',  # u10
            '10m_v_component_of_wind',  # v10
        ],
        'year': '2024',
        'month': '03',
        'day': ['15', '16', '17'],
        'time': ['00:00','06:00','12:00','18:00'],
        'area': [39.5, 124.0, 33.0, 131.0],
        'format': 'netcdf',
    },
    'era5land_korea.nc')

2025-05-12 19:10:29,698 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-12 19:10:29,700 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-12 19:10:30,430 INFO Request ID is 7cdd0937-5dfc-41fc-8712-82df966bb58f
2025-05-12 19:10:30,979 INFO status has been updated to accepted
2025-05-12 19:11:07,049 INFO status has been updated to running
2025-05-12 19:11:50,602 INFO status has been updated to successful


cb345a7d3bd70a780825b8d122dcd565.zip:   0%|          | 0.00/290k [00:00<?, ?B/s]

'era5land_korea.nc'

In [1]:
import cdsapi
import calendar
import xarray as xr
import pandas as pd
import glob
import os

def download_and_convert_month(year: str, month: str, area: list, prefix: str):
    days = [f"{d:02d}" for d in range(1, calendar.monthrange(int(year), int(month))[1] + 1)]
    times = [f"{h:02d}:00" for h in range(24)]
    c = cdsapi.Client()
    nc_file = f"{prefix}_{year}{month}.nc"
    c.retrieve(
        'reanalysis-era5-land',
        {
            'variable': [
                '2m_dewpoint_temperature',
                'total_precipitation',
                '2m_temperature',
                '10m_u_component_of_wind',
                '10m_v_component_of_wind',
            ],
            'year': year,
            'month': month,
            'day': days,
            'time': times,
            'area': area,
            'format': 'netcdf',
        },
        nc_file
    )
    print(f"Downloaded: {nc_file}")
    ds = xr.open_dataset(nc_file, engine='netcdf4')
    ds['wind10'] = (ds['10m_u_component_of_wind']**2 + ds['10m_v_component_of_wind']**2)**0.5
    df = ds[['2m_temperature','2m_dewpoint_temperature',
             'total_precipitation','wind10']].to_dataframe().reset_index()
    csv_file = f"{prefix}_{year}{month}.csv"
    df.to_csv(csv_file, index=False)
    ds.close()
    os.remove(nc_file)
    print(f"Saved CSV: {csv_file}")

if __name__ == "__main__":
    YEAR = "2024"
    PREFIX = "era5land_korea"
    AREA = [39.5, 124.0, 33.0, 131.0]

    # 월별 다운로드 & 변환
    for m in range(1, 13):
        download_and_convert_month(YEAR, f"{m:02d}", AREA, PREFIX)

    # 병합
    files = sorted(glob.glob(f"{PREFIX}_{YEAR}*.csv"))
    df_all = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)
    df_all.to_csv(f"{PREFIX}_{YEAR}_full.csv", index=False)
    print(f"Merged CSV saved: {PREFIX}_{YEAR}_full.csv")

2025-05-13 09:13:39,056 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-13 09:13:39,059 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-13 09:13:39,764 INFO Request ID is 09723bc4-92e4-4fca-92fc-63b3867ccb23
2025-05-13 09:13:40,094 INFO status has been updated to accepted
2025-05-13 09:14:03,173 INFO status has been updated to successful


5fe23c2ebccda1c3cb20eb9ada80b364.zip:   0%|          | 0.00/13.8M [00:00<?, ?B/s]

Downloaded: era5land_korea_202401.nc


ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [2]:
%pip show cdsapi

Name: cdsapi
Version: 0.7.5
Summary: Climate Data Store API
Home-page: https://github.com/ecmwf/cdsapi
Author: ECMWF
Author-email: software.support@ecmwf.int
License: Apache 2.0
Location: c:\Users\USER\anaconda3\envs\ds_env\Lib\site-packages
Requires: datapi, requests, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.
